In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os, sys, json, math, warnings, logging
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional

import numpy as np
import pandas as pd

try:
    import yfinance as yf
    import matplotlib.pyplot as plt
except Exception:
    pass

warnings.filterwarnings("ignore", category=FutureWarning)

# =========================
# LOGGING
# =========================
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
log = logging.getLogger("ema_rsi_adx_v4_eq")

# =========================
# CONFIG
# =========================
@dataclass
class Config:
    # Data
    start_date: str = "2015-01-01"
    end_date: str   = None
    static_symbols: Optional[List[str]] = None
    static_symbols_path: Optional[str] = None
    cache_dir: str = "cache"
    out_dir: str   = "outputs"
    plot: bool     = True

    # Indicators
    ema_fast: int   = 20
    ema_slow: int   = 50
    ema_htf:  int   = 200       # higher timeframe trend EMA
    use_htf_trend: bool = True  # require Close > EMA200
    rsi_len: int    = 14
    rsi_buy_max: float  = 30.0  # RSI cross up through this
    rsi_sell_min: float = 70.0

    # ADX
    adx_len: int = 14
    adx_min: float = 20.0       # require ADX > 20

    # Fixed exits
    stop_loss_pct: float = 0.02   # 2% stop
    target_pct: float    = 0.10   # 10% take profit

    # Portfolio (equal-weight; no risk/caps/cooldown)
    apply_fees: bool    = True
    initial_capital: float = 200_000.0

    # Execution
    entry_on_next_open: bool = True
    exit_on_next_open: bool = True

    # Optional guards (OFF by default)
    enable_basic_liquidity: bool = False
    min_price_inr: float = 50.0
    min_avg_vol_20d: float = 50000.0

CFG = Config()

# =========================
# FEES (per user spec)
# =========================
APPLY_FEES = True

def calc_fees(turnover_buy: float, turnover_sell: float) -> float:
    if not APPLY_FEES:
        return 0.0
    BROKER_PCT = 0.001
    BROKER_MIN = 5.0
    BROKER_CAP = 20.0
    STT_PCT = 0.001
    STAMP_BUY_PCT = 0.00015
    EXCH_PCT = 0.0000297
    SEBI_PCT = 0.000001
    IPFT_PCT = 0.000001
    GST_PCT = 0.18
    DP_SELL = 20.0 if turnover_sell >= 100 else 0.0

    def _broker(turnover):
        if turnover <= 0: return 0.0
        fee = turnover * BROKER_PCT
        fee = max(BROKER_MIN, min(fee, BROKER_CAP))
        return fee

    br_buy  = _broker(turnover_buy)
    br_sell = _broker(turnover_sell)
    stt   = STT_PCT * (turnover_buy + turnover_sell)
    stamp = STAMP_BUY_PCT * turnover_buy
    exch  = EXCH_PCT * (turnover_buy + turnover_sell)
    sebi  = SEBI_PCT * (turnover_buy + turnover_sell)
    ipft  = IPFT_PCT * (turnover_buy + turnover_sell)
    dp    = DP_SELL
    gst_base = br_buy + br_sell + dp + exch + sebi + ipft
    gst   = GST_PCT * gst_base
    return float((br_buy + br_sell) + stt + stamp + exch + sebi + ipft + dp + gst)

# =========================
# Helpers
# =========================
def ensure_dirs(*paths):
    for p in paths:
        os.makedirs(p, exist_ok=True)

def today_str():
    return pd.Timestamp.today(tz="Asia/Kolkata").strftime("%Y-%m-%d")

def load_static_symbols(static_symbols: Optional[List[str]], static_symbols_path: Optional[str]) -> List[str]:
    syms: List[str] = []
    if static_symbols and len(static_symbols) > 0:
        syms = list(static_symbols)
    elif static_symbols_path and os.path.exists(static_symbols_path):
        with open(static_symbols_path, "r") as f:
            syms = [line.strip() for line in f if line.strip()]
    else:
        raise ValueError(
            "Provide CFG.static_symbols=[...] ('.NS' suffixes) or set CFG.static_symbols_path "
            "to a file containing one symbol per line."
        )
    out = []
    for s in syms:
        s = s.strip().upper()
        if not s.endswith(".NS"):
            s = f"{s}.NS"
        out.append(s)
    seen = set(); uniq = []
    for s in out:
        if s not in seen:
            uniq.append(s); seen.add(s)
    return uniq

def fetch_prices(tickers: List[str], start: str, end: Optional[str], cache_dir: str) -> Dict[str, pd.DataFrame]:
    ensure_dirs(cache_dir)
    data = {}
    end = end or today_str()
    for ticker in tickers:
        cache_path = os.path.join(cache_dir, f"{ticker.replace('^', '_')}.parquet")
        if os.path.exists(cache_path):
            try:
                df = pd.read_parquet(cache_path)
                if len(df) and pd.to_datetime(df.index[-1]).strftime("%Y-%m-%d") >= end:
                    data[ticker] = df
                    continue
            except Exception:
                pass
        try:
            df = yf.download(ticker, start=start, end=end, auto_adjust=True, progress=False, multi_level_index=False)
            if df is None or df.empty:
                continue
            df = df.rename(columns=str.title)  # Open, High, Low, Close, Volume
            df = df[['Open', 'High', 'Low', 'Close', 'Volume']].dropna()
            df.index.name = "date"
            df.to_parquet(cache_path)
            data[ticker] = df
        except Exception:
            continue
    return data

def ema(series: pd.Series, span: int) -> pd.Series:
    return series.ewm(span=span, adjust=False, min_periods=span).mean()

def rsi(series: pd.Series, length: int = 14) -> pd.Series:
    delta = series.diff()
    gain = (delta.where(delta > 0, 0.0)).rolling(length).mean()
    loss = (-delta.where(delta < 0, 0.0)).rolling(length).mean()
    rs = gain / loss.replace(0.0, np.nan)
    out = 100 - (100 / (1 + rs))
    return out.fillna(50.0)

# --- ADX ---
def _true_range(high: pd.Series, low: pd.Series, prev_close: pd.Series) -> pd.Series:
    return pd.concat([
        (high - low).abs(),
        (high - prev_close).abs(),
        (low - prev_close).abs()
    ], axis=1).max(axis=1)

def adx(high: pd.Series, low: pd.Series, close: pd.Series, length: int = 14) -> pd.Series:
    prev_high = high.shift(1)
    prev_low  = low.shift(1)
    prev_close = close.shift(1)

    up_move   = high - prev_high
    down_move = prev_low - low
    plus_dm  = up_move.where((up_move > down_move) & (up_move > 0), 0.0)
    minus_dm = down_move.where((down_move > up_move) & (down_move > 0), 0.0)

    tr = _true_range(high, low, prev_close)

    alpha = 1.0 / length
    atr = tr.ewm(alpha=alpha, adjust=False, min_periods=length).mean()
    plus_di  = 100 * (plus_dm.ewm(alpha=alpha, adjust=False, min_periods=length).mean() / atr)
    minus_di = 100 * (minus_dm.ewm(alpha=alpha, adjust=False, min_periods=length).mean() / atr)

    dx = 100 * (plus_di - minus_di).abs() / (plus_di + minus_di).replace(0, np.nan)
    adx_series = dx.ewm(alpha=alpha, adjust=False, min_periods=length).mean()
    return adx_series

def compute_indicators(df: pd.DataFrame, cfg: Config) -> pd.DataFrame:
    out = df.copy()
    out["ema_fast"] = ema(out["Close"], cfg.ema_fast)
    out["ema_slow"] = ema(out["Close"], cfg.ema_slow)
    out["ema_htf"]  = ema(out["Close"], cfg.ema_htf)
    out["rsi"]      = rsi(out["Close"], cfg.rsi_len)
    out["rsi_prev"] = out["rsi"].shift(1)
    out["avg_vol_20"] = out["Volume"].rolling(20).mean()
    out["adx"]      = adx(out["High"], out["Low"], out["Close"], cfg.adx_len)
    out["trend_strength"] = (out["ema_fast"] - out["ema_slow"]) / out["Close"]
    return out.dropna()

def basic_liquidity_ok(row: pd.Series, cfg: Config) -> bool:
    if not cfg.enable_basic_liquidity:
        return True
    if row["Close"] < cfg.min_price_inr:
        return False
    if row["avg_vol_20"] < cfg.min_avg_vol_20d:
        return False
    return True

def entry_score(row: pd.Series) -> float:
    return float(1000.0 * row.get("trend_strength", 0.0) + row.get("adx", 0.0) - row.get("rsi", 50.0))

def simulate_ticker(ticker: str, df: pd.DataFrame, cfg: Config):
    d = compute_indicators(df, cfg).copy()
    if d.empty:
        return pd.DataFrame(columns=["ticker","side","date","price","shares","reason","pnl_inr"]), pd.Series(dtype=float)

    trend_ok = (d["ema_fast"] > d["ema_slow"])
    htf_ok   = (d["Close"] > d["ema_htf"]) if cfg.use_htf_trend else pd.Series(True, index=d.index)
    rsi_cross = (d["rsi_prev"] < cfg.rsi_buy_max) & (d["rsi"] >= cfg.rsi_buy_max)
    adx_ok  = (d["adx"] > cfg.adx_min)

    entry_signal = trend_ok & htf_ok & rsi_cross & adx_ok
    exit_ind = (d["rsi"] > cfg.rsi_sell_min) | (d["ema_fast"] < d["ema_slow"])

    in_pos = False
    entry_px = 0.0
    stop_px  = 0.0
    tgt_px   = 0.0

    trades = []

    idx = list(d.index)
    for i in range(len(idx)-1):
        dt = idx[i]
        nxt = idx[i+1]
        row = d.loc[dt]
        nxt_row = d.loc[nxt]

        if not in_pos:
            if entry_signal.loc[dt] and basic_liquidity_ok(row, cfg):
                gap = (nxt_row["Open"] - row["Close"]) / row["Close"]
                if abs(gap) <= 0.02:
                    px = float(nxt_row["Open"] if cfg.entry_on_next_open else row["Close"])
                    trades.append({
                        "ticker": ticker, "side": "BUY", "date": nxt if cfg.entry_on_next_open else dt,
                        "price": px, "shares": 0, "reason": "rsi_cross_up & adx>min & trend(ema & ema200)",
                        "score": entry_score(row)
                    })
                    in_pos = True
                    entry_px = px
                    stop_px = entry_px * (1 - cfg.stop_loss_pct)
                    tgt_px  = entry_px * (1 + cfg.target_pct)
        else:
            hit = None
            exec_date = nxt if cfg.exit_on_next_open else dt
            exec_price = None

            if nxt_row["Low"] <= stop_px and nxt_row["High"] >= tgt_px:
                hit = "target"
                exec_price = float(tgt_px)
            elif nxt_row["Low"] <= stop_px:
                hit = "stop"
                exec_price = float(stop_px)
            elif nxt_row["High"] >= tgt_px:
                hit = "target"
                exec_price = float(tgt_px)
            else:
                if exit_ind.loc[dt]:
                    exec_price = float(nxt_row["Open"] if cfg.exit_on_next_open else row["Close"])
                    hit = "indicator_exit"

            if hit is not None:
                trades.append({
                    "ticker": ticker, "side": "SELL", "date": exec_date,
                    "price": float(exec_price), "shares": 0, "reason": hit, "pnl_inr": 0.0
                })
                in_pos = False
                entry_px = stop_px = tgt_px = 0.0

    if in_pos:
        last_dt = d.index[-1]
        last_close = float(d.loc[last_dt, "Close"])
        trades.append({
            "ticker": ticker, "side": "SELL", "date": last_dt,
            "price": last_close, "shares": 0, "reason": "final_close", "pnl_inr": 0.0
        })

    trades_df = pd.DataFrame(trades)
    return trades_df, pd.Series(dtype=float)

def aggregate_and_apply_fees(all_trades: pd.DataFrame, data_map: Dict[str, pd.DataFrame], cfg: Config):
    if all_trades.empty:
        return all_trades, pd.Series(dtype=float), {}

    side_order = {"BUY": 0, "SELL": 1}
    order = all_trades["side"].map(side_order)
    all_trades = (all_trades
        .assign(_sorder=order)
        .sort_values(by=["date", "_sorder"], kind="stable")
        .drop(columns=["_sorder"])
        .reset_index(drop=True)
    )
    all_trades["date"] = pd.to_datetime(all_trades["date"])

    # Prepare equity (cash + MTM) with seed
    equity_curve = []
    dates = sorted(all_trades["date"].unique().tolist())
    cash = cfg.initial_capital
    open_positions = {}
    completed_legs = []

    global APPLY_FEES
    APPLY_FEES = cfg.apply_fees

    def _get_close_on(tkr, dt):
        df = data_map.get(tkr)
        if df is None or df.empty: return np.nan
        if dt in df.index: return float(df.loc[dt, "Close"])
        prev = df[df.index <= dt]
        if prev.empty: return np.nan
        return float(prev["Close"].iloc[-1])

    # Seed initial equity point (day before first trade)
    if dates:
        seed_date = pd.to_datetime(dates[0]) - pd.Timedelta(days=1)
        equity_curve.append((seed_date, float(cash)))

    for dt in dates:
        day_trades = all_trades[all_trades["date"] == dt].copy()

        # ---- RANK (informational) ----
        buy_cands = day_trades[day_trades["side"]=="BUY"].copy()
        if not buy_cands.empty:
            buy_cands["score"] = buy_cands["score"].fillna(0.0)
            buy_cands = buy_cands.sort_values("score", ascending=False)
            log.info("Ranking %d BUY signals on %s (equal-weight; selecting ALL):", len(buy_cands), dt.date())
            for _, rr in buy_cands.iterrows():
                log.info("  %-12s score=%6.2f  price=%8.2f  rsi=%5.1f  adx=%5.1f",
                         rr["ticker"], rr["score"], rr["price"], rr.get("rsi", np.nan), rr.get("adx", np.nan))

        # ---- SELL first (free up cash) ----
        for _, tr in day_trades[day_trades["side"] == "SELL"].iterrows():
            tkr = tr["ticker"]; price = float(tr["price"])
            pos = open_positions.get(tkr)
            if pos is None:
                continue
            shares = int(pos["shares"])
            turnover_sell = shares * price
            fee = calc_fees(0.0, turnover_sell)
            pnl = (price - pos["entry_px"]) * shares
            cash += (turnover_sell - fee)
            realized = pnl - fee - pos.get("buy_fee", 0.0)
            completed_legs.append({
                "ticker": tkr, "side": "SELL", "date": dt, "price": price,
                "shares": shares, "reason": tr.get("reason",""),
                "turnover": turnover_sell, "fees_inr": fee, "pnl_inr": realized
            })
            log.info("Exit %-12s px=%8.2f shares=%6d reason=%s net=%.2f cash=%.2f",
                     tkr, price, shares, tr.get("reason",""), realized, cash)
            del open_positions[tkr]

        # ---- BUYs (equal allocation of available cash across ALL buys today) ----
        buys_today = day_trades[day_trades["side"] == "BUY"].copy()
        n_buys = buys_today.shape[0]
        if n_buys > 0 and cash > 0:
            per_alloc = cash / n_buys
            for _, tr in buys_today.iterrows():
                tkr = tr["ticker"]; price = float(tr["price"])
                shares = int(math.floor(per_alloc / price))
                if shares <= 0:
                    log.info("Skip BUY %-12s px=%8.2f (alloc %.2f too small for 1 share)", tkr, price, per_alloc)
                    continue
                turnover_buy = shares * price
                fee = calc_fees(turnover_buy, 0.0)
                total_cost = turnover_buy + fee
                if total_cost > cash:
                    shares = int(math.floor((cash - fee) / price))
                    if shares <= 0:
                        log.info("Skip BUY %-12s due to fees/cash constraint", tkr)
                        continue
                    turnover_buy = shares * price
                    total_cost = turnover_buy + fee
                cash -= total_cost
                open_positions[tkr] = {
                    "entry_date": dt, "entry_px": price, "shares": shares,
                    "buy_fee": fee, "entry_reason": tr.get("reason",""),
                    "score": float(tr.get("score", 0.0))
                }
                completed_legs.append({
                    "ticker": tkr, "side": "BUY", "date": dt, "price": price,
                    "shares": shares, "reason": tr.get("reason",""),
                    "turnover": turnover_buy, "fees_inr": fee, "pnl_inr": 0.0
                })
                log.info("Selected BUY (EQ): %-12s px=%8.2f shares=%6d fee=%.2f cash=%.2f", tkr, price, shares, fee, cash)

        # MTM valuation
        mtm = 0.0
        for _tkr, pos in open_positions.items():
            px = _get_close_on(_tkr, dt)
            if not np.isnan(px):
                mtm += pos["shares"] * px
        total_equity = cash + mtm
        equity_curve.append((dt, float(total_equity)))

    eq_ser = pd.Series([e for _, e in equity_curve], index=[d for d, _ in equity_curve])
    legs_df = pd.DataFrame(completed_legs).sort_values(["date", "ticker", "side"]).reset_index(drop=True)

    # Build roundtrips
    roundtrips = []
    by_tkr_open = {}
    for _, leg in legs_df.iterrows():
        tkr = leg["ticker"]
        if leg["side"] == "BUY":
            by_tkr_open[tkr] = leg
        else:
            buy = by_tkr_open.pop(tkr, None)
            if buy is None:
                continue
            fees_total = float(buy.get("fees_inr", 0.0) + leg.get("fees_inr", 0.0))
            gross_pnl = (leg["price"] - buy["price"]) * buy["shares"]
            net_pnl   = gross_pnl - fees_total
            ret_pct   = (leg["price"] / buy["price"] - 1.0) * 100.0
            days_held = (pd.to_datetime(leg["date"]) - pd.to_datetime(buy["date"])).days
            roundtrips.append({
                "ticker": tkr,
                "entry_date": pd.to_datetime(buy["date"]),
                "entry_price": float(buy["price"]),
                "exit_date": pd.to_datetime(leg["date"]),
                "exit_price": float(leg["price"]),
                "days_held": int(days_held),
                "shares": int(buy["shares"]),
                "entry_reason": buy.get("reason",""),
                "exit_reason": leg.get("reason",""),
                "gross_pnl_inr": float(gross_pnl),
                "fees_total_inr": float(fees_total),
                "net_pnl_inr": float(net_pnl),
                "return_pct": float(ret_pct),
                "rank_score": float(buy.get("score", 0.0))
            })
    trips_df = pd.DataFrame(roundtrips).sort_values(["entry_date","ticker"]).reset_index(drop=True)

    metrics = compute_metrics(eq_ser, legs_df)
    return legs_df, trips_df, eq_ser, metrics

def compute_metrics(equity: pd.Series, legs_df: pd.DataFrame):
    out = {}
    if equity is None or equity.empty:
        return out
    eq = equity.dropna()
    daily_ret = eq.pct_change().fillna(0.0)

    days = (eq.index[-1] - eq.index[0]).days or 1
    years = days / 365.25
    cagr = (eq.iloc[-1] / eq.iloc[0]) ** (1/years) - 1 if years > 0 else 0.0

    if daily_ret.std(ddof=0) > 0:
        sharpe = (daily_ret.mean() / daily_ret.std(ddof=0)) * np.sqrt(252)
    else:
        sharpe = 0.0

    cummax = eq.cummax()
    dd = (eq - cummax) / cummax
    max_dd = dd.min()

    wins = 0
    n_sells = legs_df[legs_df["side"] == "SELL"].shape[0] if legs_df is not None and not legs_df.empty else 0
    for _, r in legs_df[legs_df["side"] == "SELL"].iterrows():
        if float(r["pnl_inr"]) > 0: wins += 1
    win_rate = (wins / n_sells) * 100.0 if n_sells > 0 else 0.0

    out.update({
        "start_equity_inr": float(eq.iloc[0]),
        "final_equity_inr": float(eq.iloc[-1]),
        "cagr_pct": float(cagr * 100),
        "sharpe": float(sharpe),
        "max_drawdown_pct": float(max_dd * 100),
        "win_rate_pct": float(win_rate),
        "n_trades": int(n_sells),
    })
    return out

def plot_equity(equity: pd.Series, out_path: str):
    if equity is None or equity.empty:
        return
    import matplotlib.pyplot as plt
    plt.figure(figsize=(10,5))
    plt.plot(equity.index, equity.values)
    plt.title("Equity Curve")
    plt.xlabel("Date")
    plt.ylabel("Equity (INR)")
    plt.tight_layout()
    plt.savefig(out_path)
    plt.close()

def backtest(cfg: Config):
    ensure_dirs(cfg.cache_dir, cfg.out_dir)
    log.info("Universe: loading static symbols...")
    symbols = load_static_symbols(cfg.static_symbols, cfg.static_symbols_path)
    log.info("Loaded %d symbols.", len(symbols))

    log.info("Data: fetching OHLCV from yfinance (adjusted)...")
    data_map = fetch_prices(symbols, cfg.start_date, cfg.end_date, cfg.cache_dir)
    log.info("Downloaded %d symbols with data.", len(data_map))

    log.info("Signals: generating EMA/RSI/ADX and candidate entries...")
    all_trades = []
    for i, tkr in enumerate(symbols, 1):
        df = data_map.get(tkr)
        if df is None or df.empty:
            continue
        tr, _ = simulate_ticker(tkr, df, cfg)
        if not tr.empty: all_trades.append(tr)
        if i % 50 == 0:
            log.info("  processed %d/%d tickers...", i, len(symbols))

    if not all_trades:
        log.warning("No signals generated; check your thresholds.")
        return None, None, None, {}
    all_trades = pd.concat(all_trades, ignore_index=True)

    log.info("Portfolio: equal-weight BUY allocation per day; applying fees...")
    legs_df, trips_df, equity, metrics = aggregate_and_apply_fees(all_trades, data_map, cfg)

    stamp = pd.Timestamp.today(tz="Asia/Kolkata").strftime("%Y%m%d_%H%M%S")
    legs_path = os.path.join(cfg.out_dir, f"trades_legs_{stamp}.csv")
    trips_path = os.path.join(cfg.out_dir, f"trades_roundtrips_{stamp}.csv")
    equity_path = os.path.join(cfg.out_dir, f"equity_{stamp}.csv")
    metrics_path = os.path.join(cfg.out_dir, f"metrics_{stamp}.json")
    eq_plot_path = os.path.join(cfg.out_dir, f"equity_{stamp}.png")

    if legs_df is not None:
        legs_df.to_csv(legs_path, index=False)
    if trips_df is not None:
        trips_df.to_csv(trips_path, index=False)
    if equity is not None:
        pd.DataFrame({"date": equity.index, "equity": equity.values}).to_csv(equity_path, index=False)
    with open(metrics_path, "w") as f:
        json.dump(metrics, f, indent=2)

    if cfg.plot and equity is not None:
        plot_equity(equity, eq_plot_path)

    log.info("=== METRICS ===\n%s", json.dumps(metrics, indent=2))
    log.info("Files written:\n  %s\n  %s\n  %s\n  %s", legs_path, trips_path, equity_path, metrics_path)
    if cfg.plot:
        log.info("  %s", eq_plot_path)

def main():
    global APPLY_FEES
    APPLY_FEES = bool(CFG.apply_fees)
    backtest(CFG)

if __name__ == "__main__":
    # Example: set your universe here or via file path.
    CFG.static_symbols = ['360ONE.NS', '3MINDIA.NS', 'AADHARHFC.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABB.NS', 'ABBOTINDIA.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ABLBL.NS', 'ABREL.NS', 'ABSLAMC.NS', 'ACC.NS', 'ACE.NS', 'ACMESOLAR.NS', 'ADANIENSOL.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'AEGISLOG.NS', 'AEGISVOPAK.NS', 'AFCONS.NS', 'AFFLE.NS', 'AGARWALEYE.NS', 'AIAENG.NS', 'AIIL.NS', 'AJANTPHARM.NS', 'AKUMS.NS', 'AKZOINDIA.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALOKINDS.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANANDRATHI.NS', 'ANANTRAJ.NS', 'ANGELONE.NS', 'APARINDS.NS', 'APLAPOLLO.NS', 'APLLTD.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'APTUS.NS', 'ARE&M.NS', 'ASAHIINDIA.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'ASTERDM.NS', 'ASTRAL.NS', 'ASTRAZEN.NS', 'ATGL.NS', 'ATHERENERG.NS', 'ATUL.NS', 'AUBANK.NS', 'AUROPHARMA.NS', 'AWL.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJAJFINSV.NS', 'BAJAJHFL.NS', 'BAJAJHLDNG.NS', 'BAJFINANCE.NS', 'BALKRISIND.NS', 'BALRAMCHIN.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BANKINDIA.NS', 'BASF.NS', 'BATAINDIA.NS', 'BAYERCROP.NS', 'BBTC.NS', 'BDL.NS', 'BEL.NS', 'BEML.NS', 'BERGEPAINT.NS', 'BHARATFORG.NS', 'BHARTIARTL.NS', 'BHARTIHEXA.NS', 'BHEL.NS', 'BIKAJI.NS', 'BIOCON.NS', 'BLS.NS', 'BLUEDART.NS', 'BLUEJET.NS', 'BLUESTARCO.NS', 'BOSCHLTD.NS', 'BPCL.NS', 'BRIGADE.NS', 'BRITANNIA.NS', 'BSE.NS', 'BSOFT.NS', 'CAMPUS.NS', 'CAMS.NS', 'CANBK.NS', 'CANFINHOME.NS', 'CAPLIPOINT.NS', 'CARBORUNIV.NS', 'CASTROLIND.NS', 'CCL.NS', 'CDSL.NS', 'CEATLTD.NS', 'CENTRALBK.NS', 'CENTURYPLY.NS', 'CERA.NS', 'CESC.NS', 'CGCL.NS', 'CGPOWER.NS', 'CHALET.NS', 'CHAMBLFERT.NS', 'CHENNPETRO.NS', 'CHOICEIN.NS', 'CHOLAFIN.NS', 'CHOLAHLDNG.NS', 'CIPLA.NS', 'CLEAN.NS', 'COALINDIA.NS', 'COCHINSHIP.NS', 'COFORGE.NS', 'COHANCE.NS', 'COLPAL.NS', 'CONCOR.NS', 'CONCORDBIO.NS', 'COROMANDEL.NS', 'CRAFTSMAN.NS', 'CREDITACC.NS', 'CRISIL.NS', 'CROMPTON.NS', 'CUB.NS', 'CUMMINSIND.NS', 'CYIENT.NS', 'DABUR.NS', 'DALBHARAT.NS', 'DATAPATTNS.NS', 'DBREALTY.NS', 'DCMSHRIRAM.NS', 'DEEPAKFERT.NS', 'DEEPAKNTR.NS', 'DELHIVERY.NS', 'DEVYANI.NS', 'DIVISLAB.NS', 'DIXON.NS', 'DLF.NS', 'DMART.NS', 'DOMS.NS', 'DRREDDY.NS', 'ECLERX.NS', 'EICHERMOT.NS', 'EIDPARRY.NS', 'EIHOTEL.NS', 'ELECON.NS', 'ELGIEQUIP.NS', 'EMAMILTD.NS', 'EMCURE.NS', 'ENDURANCE.NS', 'ENGINERSIN.NS', 'ENRIN.NS', 'ERIS.NS', 'ESCORTS.NS', 'ETERNAL.NS', 'EXIDEIND.NS', 'FACT.NS', 'FEDERALBNK.NS', 'FINCABLES.NS', 'FINPIPE.NS', 'FIRSTCRY.NS', 'FIVESTAR.NS', 'FLUOROCHEM.NS', 'FORCEMOT.NS', 'FORTIS.NS', 'FSL.NS', 'GAIL.NS', 'GESHIP.NS', 'GICRE.NS', 'GILLETTE.NS', 'GLAND.NS', 'GLAXO.NS', 'GLENMARK.NS', 'GMDCLTD.NS', 'GMRAIRPORT.NS', 'GODFRYPHLP.NS', 'GODIGIT.NS', 'GODREJAGRO.NS', 'GODREJCP.NS', 'GODREJIND.NS', 'GODREJPROP.NS', 'GPIL.NS', 'GRANULES.NS', 'GRAPHITE.NS', 'GRASIM.NS', 'GRAVITA.NS', 'GRSE.NS', 'GSPL.NS', 'GUJGASLTD.NS', 'GVT&D.NS', 'HAL.NS', 'HAPPSTMNDS.NS', 'HAVELLS.NS', 'HBLENGINE.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HEG.NS', 'HEROMOTOCO.NS', 'HEXT.NS', 'HFCL.NS', 'HINDALCO.NS', 'HINDCOPPER.NS', 'HINDPETRO.NS', 'HINDUNILVR.NS', 'HINDZINC.NS', 'HOMEFIRST.NS', 'HONASA.NS', 'HONAUT.NS', 'HSCL.NS', 'HUDCO.NS', 'HYUNDAI.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'IDBI.NS', 'IDEA.NS', 'IDFCFIRSTB.NS', 'IEX.NS', 'IFCI.NS', 'IGIL.NS', 'IGL.NS', 'IIFL.NS', 'IKS.NS', 'INDGN.NS', 'INDHOTEL.NS', 'INDIACEM.NS', 'INDIAMART.NS', 'INDIANB.NS', 'INDIGO.NS', 'INDUSINDBK.NS', 'INDUSTOWER.NS', 'INFY.NS', 'INOXINDIA.NS', 'INOXWIND.NS', 'INTELLECT.NS', 'IOB.NS', 'IOC.NS', 'IPCALAB.NS', 'IRB.NS', 'IRCON.NS', 'IRCTC.NS', 'IREDA.NS', 'IRFC.NS', 'ITC.NS', 'ITCHOTELS.NS', 'ITI.NS', 'J&KBANK.NS', 'JBCHEPHARM.NS', 'JBMA.NS', 'JINDALSAW.NS', 'JINDALSTEL.NS', 'JIOFIN.NS', 'JKCEMENT.NS', 'JKTYRE.NS', 'JMFINANCIL.NS', 'JPPOWER.NS', 'JSL.NS', 'JSWENERGY.NS', 'JSWINFRA.NS', 'JSWSTEEL.NS', 'JUBLFOOD.NS', 'JUBLINGREA.NS', 'JUBLPHARMA.NS', 'JWL.NS', 'JYOTHYLAB.NS', 'JYOTICNC.NS', 'KAJARIACER.NS', 'KALYANKJIL.NS', 'KARURVYSYA.NS', 'KAYNES.NS', 'KEC.NS', 'KEI.NS', 'KFINTECH.NS', 'KIMS.NS', 'KIRLOSBROS.NS', 'KIRLOSENG.NS', 'KOTAKBANK.NS', 'KPIL.NS', 'KPITTECH.NS', 'KPRMILL.NS', 'KSB.NS', 'LALPATHLAB.NS', 'LATENTVIEW.NS', 'LAURUSLABS.NS', 'LEMONTREE.NS', 'LICHSGFIN.NS', 'LICI.NS', 'LINDEINDIA.NS', 'LLOYDSME.NS', 'LODHA.NS', 'LT.NS', 'LTF.NS', 'LTFOODS.NS', 'LTIM.NS', 'LTTS.NS', 'LUPIN.NS', 'M&M.NS', 'M&MFIN.NS', 'MAHABANK.NS', 'MAHSCOOTER.NS', 'MAHSEAMLES.NS', 'MANAPPURAM.NS', 'MANKIND.NS', 'MANYAVAR.NS', 'MAPMYINDIA.NS', 'MARICO.NS', 'MARUTI.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'MCX.NS', 'MEDANTA.NS', 'METROPOLIS.NS', 'MFSL.NS', 'MGL.NS', 'MINDACORP.NS', 'MMTC.NS', 'MOTHERSON.NS', 'MOTILALOFS.NS', 'MPHASIS.NS', 'MRF.NS', 'MRPL.NS', 'MSUMI.NS', 'MUTHOOTFIN.NS', 'NAM-INDIA.NS', 'NATCOPHARM.NS', 'NATIONALUM.NS', 'NAUKRI.NS', 'NAVA.NS', 'NAVINFLUOR.NS', 'NBCC.NS', 'NCC.NS', 'NESTLEIND.NS', 'NETWEB.NS', 'NEULANDLAB.NS', 'NEWGEN.NS', 'NH.NS', 'NHPC.NS', 'NIACL.NS', 'NIVABUPA.NS', 'NLCINDIA.NS', 'NMDC.NS', 'NSLNISP.NS', 'NTPC.NS', 'NTPCGREEN.NS', 'NUVAMA.NS', 'NUVOCO.NS', 'NYKAA.NS', 'OBEROIRLTY.NS', 'OFSS.NS', 'OIL.NS', 'OLAELEC.NS', 'OLECTRA.NS', 'ONESOURCE.NS', 'ONGC.NS', 'PAGEIND.NS', 'PATANJALI.NS', 'PAYTM.NS', 'PCBL.NS', 'PERSISTENT.NS', 'PETRONET.NS', 'PFC.NS', 'PFIZER.NS', 'PGEL.NS', 'PGHH.NS', 'PHOENIXLTD.NS', 'PIDILITIND.NS', 'PIIND.NS', 'PNB.NS', 'PNBHOUSING.NS', 'POLICYBZR.NS', 'POLYCAB.NS', 'POLYMED.NS', 'POONAWALLA.NS', 'POWERGRID.NS', 'POWERINDIA.NS', 'PPLPHARMA.NS', 'PRAJIND.NS', 'PREMIERENE.NS', 'PRESTIGE.NS', 'PTCIL.NS', 'PVRINOX.NS', 'RADICO.NS', 'RAILTEL.NS', 'RAINBOW.NS', 'RAMCOCEM.NS', 'RBLBANK.NS', 'RCF.NS', 'RECLTD.NS', 'REDINGTON.NS', 'RELIANCE.NS', 'RELINFRA.NS', 'RHIM.NS', 'RITES.NS', 'RKFORGE.NS', 'RPOWER.NS', 'RRKABEL.NS', 'RVNL.NS', 'SAGILITY.NS', 'SAIL.NS', 'SAILIFE.NS', 'SAMMAANCAP.NS', 'SAPPHIRE.NS', 'SARDAEN.NS', 'SAREGAMA.NS', 'SBFC.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SBIN.NS', 'SCHAEFFLER.NS', 'SCHNEIDER.NS', 'SCI.NS', 'SHREECEM.NS', 'SHRIRAMFIN.NS', 'SHYAMMETL.NS', 'SIEMENS.NS', 'SIGNATURE.NS', 'SJVN.NS', 'SKFINDIA.NS', 'SOBHA.NS', 'SOLARINDS.NS', 'SONACOMS.NS', 'SONATSOFTW.NS', 'SRF.NS', 'STARHEALTH.NS', 'SUMICHEM.NS', 'SUNDARMFIN.NS', 'SUNDRMFAST.NS', 'SUNPHARMA.NS', 'SUNTV.NS', 'SUPREMEIND.NS', 'SUZLON.NS', 'SWANCORP.NS', 'SWIGGY.NS', 'SYNGENE.NS', 'SYRMA.NS', 'TARIL.NS', 'TATACHEM.NS', 'TATACOMM.NS', 'TATACONSUM.NS', 'TATAELXSI.NS', 'TATAINVEST.NS', 'TATAMOTORS.NS', 'TATAPOWER.NS', 'TATASTEEL.NS', 'TATATECH.NS', 'TBOTEK.NS', 'TCS.NS', 'TECHM.NS', 'TECHNOE.NS', 'TEJASNET.NS', 'THELEELA.NS', 'THERMAX.NS', 'TIINDIA.NS', 'TIMKEN.NS', 'TITAGARH.NS', 'TITAN.NS', 'TORNTPHARM.NS', 'TORNTPOWER.NS', 'TRENT.NS', 'TRIDENT.NS', 'TRITURBINE.NS', 'TRIVENI.NS', 'TTML.NS', 'TVSMOTOR.NS', 'UBL.NS', 'UCOBANK.NS', 'ULTRACEMCO.NS', 'UNIONBANK.NS', 'UNITDSPR.NS', 'UNOMINDA.NS', 'UPL.NS', 'USHAMART.NS', 'UTIAMC.NS', 'VBL.NS', 'VEDL.NS', 'VENTIVE.NS', 'VGUARD.NS', 'VIJAYA.NS', 'VMM.NS', 'VOLTAS.NS', 'VTL.NS', 'WAAREEENER.NS', 'WELCORP.NS', 'WELSPUNLIV.NS', 'WHIRLPOOL.NS', 'WIPRO.NS', 'WOCKPHARMA.NS', 'YESBANK.NS', 'ZEEL.NS', 'ZENSARTECH.NS', 'ZENTEC.NS', 'ZFCVINDIA.NS', 'ZYDUSLIFE.NS']

    # CFG.static_symbols_path = "nifty500.txt"
    main()


2025-10-06 23:30:34 | INFO | Universe: loading static symbols...
2025-10-06 23:30:34 | INFO | Loaded 500 symbols.
2025-10-06 23:30:34 | INFO | Data: fetching OHLCV from yfinance (adjusted)...
2025-10-06 23:33:41 | INFO | Downloaded 500 symbols with data.
2025-10-06 23:33:41 | INFO | Signals: generating EMA/RSI/ADX and candidate entries...
2025-10-06 23:33:45 | INFO |   processed 50/500 tickers...
2025-10-06 23:33:49 | INFO |   processed 100/500 tickers...
2025-10-06 23:33:53 | INFO |   processed 150/500 tickers...
2025-10-06 23:33:57 | INFO |   processed 200/500 tickers...
2025-10-06 23:34:01 | INFO |   processed 250/500 tickers...
2025-10-06 23:34:05 | INFO |   processed 300/500 tickers...
2025-10-06 23:34:09 | INFO |   processed 350/500 tickers...
2025-10-06 23:34:13 | INFO |   processed 400/500 tickers...
2025-10-06 23:34:17 | INFO |   processed 450/500 tickers...
2025-10-06 23:34:21 | INFO |   processed 500/500 tickers...
2025-10-06 23:34:21 | INFO | Portfolio: equal-weight BUY all